In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/myOTP/sf

Mounted at /content/gdrive
/content/gdrive/MyDrive/myOTP/sf


In [2]:
!pip install gsw

In [3]:
import os
from typing import Optional
import numpy as np
import xarray as xr
from gsw.conversions import CT_from_pt
from gsw.density import sigma2
from geopy.distance import distance
from scipy.integrate import simpson
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pickle

In [4]:
def lat_depth(v: np.ndarray,
              x_Z: np.ndarray,
              x_lon: np.ndarray,
              d: np.ndarray,
              lat: int) -> np.ndarray:
    _, n_lons, _, n_times = v.shape
    ix = list(x_Z).index(d[lat])+1
    moc = np.empty(n_times)
    for t in range(n_times):
        # inner integral over depth
        inner = simpson(y=np.nan_to_num(v[lat, :, :ix, t]), x=x_Z[:ix], axis=1)
        # outer integral over longitude
        moc[t] = -simpson(y=inner, x=x_lon[lat, :]) / 1e6
    return moc

def lat_density(v: np.ndarray,
                x_Z: np.ndarray,
                x_lon: np.ndarray,
                lighter: np.ndarray,
                empty: np.ndarray,
                isopycnals: np.ndarray,
                lat: int) -> np.ndarray:
    _, n_lons, _, n_times = v.shape
    moc = np.empty(n_times)

    for t in range(n_times):
        for l in range(n_lons):
            if not (lighter[lat, l, t] or empty[lat, l, t]):
                ix = isopycnals[lat, l, t]
            else:
                ix = 0
            v[lat, l, ix:, t] = 0.

        # inner integral over depth
        inner = simpson(y=v[lat, :, :, t], x=x_Z, axis=1)
        # outer integral over longitude
        moc[t] = -simpson(y=inner, x=x_lon[lat, :]) / 1e6
    return moc

def psi(d: float|np.ndarray,
        v: np.ndarray,
        x_lon: np.ndarray,
        x_Z: np.ndarray,
        s2: Optional[np.ndarray],
        use_density: bool=False) -> np.ndarray:
    n_lats, _, n_depths, _ = v.shape
    d = np.array([d]*n_lats) if np.isscalar(d) else np.asarray(d)
    assert len(d) == n_lats
    if use_density:
        # find water columns lighter than d
        lighter = np.less_equal(s2[:, :, 0, :], d[:, np.newaxis, np.newaxis])
        # find missing water columns
        empty = np.isnan(s2).all(axis=2)
        # find isopycnals (defaults to 0)
        isopycnals = np.argmax(np.less_equal(s2, d[:, np.newaxis, np.newaxis, np.newaxis]), axis=2)
        # any defaults (0 values) represent entire water columns which are heavier than d
        isopycnals[isopycnals == 0] = n_depths
        f = lat_density
        _args = [v.copy(), x_Z, x_lon, lighter, empty, isopycnals]
    else:
        f = lat_depth
        _args = [v, x_Z, x_lon, d]

    # parallelise over latitudes
    # NOTE: in my testing, I found that parallelising over latitudes was more efficient than any other dimension
    with Pool(cpu_count()) as pool:
            results = pool.starmap(f, [(*_args, lat) for lat in range(n_lats)])
    return np.array(results)

In [5]:
use_bolus = True
use_density = True
density_precision = 100
data_path = "../../GTC/ecco_data"

sections = ["26N", "30S", "55S", "60S"]
coordinates = ["latitude", "longitude", "Z", "time"]

In [6]:
section = "southern_ocean"
print("fetching monthly mean velocities")
vm = xr.open_mfdataset(f"{data_path}_full/{section}/ECCO_L4_OCEAN_VEL_05DEG_MONTHLY_V4R4/*.nc",
                    coords="minimal",
                    data_vars="minimal",
                    parallel=True, compat="override")
vm = vm[["NVEL"]].transpose(*coordinates).isel(Z=slice(None, None, -1)).fillna(0.)
vm = vm.rename({"NVEL": "vm"})
if use_bolus:
    print("fetching bolus velocities")
    ve = xr.open_mfdataset(f"{data_path}_full/{section}/ECCO_L4_BOLUS_05DEG_MONTHLY_V4R4/*.nc",
                        coords="minimal",
                        data_vars="minimal",
                        parallel=True, compat="override")
    ve = ve[["NVELSTAR"]].transpose(*coordinates).isel(Z=slice(None, None, -1)).fillna(0.)
    ve = ve.rename({"NVELSTAR": "ve"})
if use_density:
    print("using density: fetching temperature and salinity for calculation")
    density = xr.open_mfdataset(f"{data_path}_full/{section}/ECCO_L4_TEMP_SALINITY_05DEG_MONTHLY_V4R4/*.nc",
                                coords="minimal",
                                data_vars="minimal",
                                parallel=True, compat="override")
    density = density[["THETA", "SALT"]].transpose(*coordinates).isel(Z=slice(None, None, -1))
    ct = CT_from_pt(SA=density["SALT"], pt=density["THETA"])
    s2 = sigma2(SA=density["SALT"], CT=ct).to_dataset()
    s2 = s2.rename({list(s2.data_vars)[0]: "sigma_2"})
v = vm["vm"] + ve["ve"] if use_bolus else vm["vm"]

grid = np.array([[(lat, lon) for lon in v["longitude"].to_numpy()] for lat in v["latitude"].to_numpy()])
# get x-coordinates of longitude measurements
x_lon = np.array([[0.]+[distance(latitude[i+1], latitude[i]).meters
                        for i in range(grid.shape[1]-1)]
                        for latitude in grid])
# rounding to cm
x_lon = np.round(np.cumsum(x_lon, -1), 2)
# get x-coordinates of depth measurements
x_Z = v["Z"].to_numpy()
# unsure if -ve is a problem but getting rid of them just in case
x_Z += max(abs(x_Z))

v_np = v.to_numpy()
if use_density: s2_np = s2["sigma_2"].to_numpy()

fetching monthly mean velocities
fetching bolus velocities
using density: fetching temperature and salinity for calculation


In [7]:
for i in range(19, 60):
    outpath = f"so/sf_{i}"
    outpath = outpath + "_density.pickle" if use_density else outpath + "_depth.pickle"
    if os.path.exists(outpath): continue
    print(f"saving to {outpath}")
    if use_density:
        temp = s2_np[i][~np.isnan(s2_np[i])]
        if len(temp) == 0: continue
        sf_range = np.linspace(min(temp)-0.1, max(temp)+0.1, density_precision)
        outfile = open(f"so/density_range_{i}.pickle", "wb")
        pickle.dump(sf_range, outfile); outfile.close()
    else:
        s2_np = None
        sf_range = x_Z

    args = [v_np[i:i+1], x_lon[i:i+1], x_Z, s2_np[i:i+1], use_density]
    # calculate the streamfunction at all possible densities/depths
    if use_density: print("calculating streamfunction for all densities")
    else: print("calculating streamfunction for all depths")
    streamfunction = np.array([psi(d, *args) for d in tqdm(sf_range)])
    outfile = open(outpath, "wb")
    pickle.dump(streamfunction, outfile); outfile.close()

saving to so/sf_42_density.pickle
calculating streamfunction for all densities


100%|██████████| 100/100 [02:33<00:00,  1.54s/it]


saving to so/sf_43_density.pickle
calculating streamfunction for all densities


100%|██████████| 100/100 [02:33<00:00,  1.53s/it]


saving to so/sf_44_density.pickle
calculating streamfunction for all densities


...

100%|██████████| 100/100 [02:48<00:00,  1.69s/it]


saving to so/sf_58_density.pickle
calculating streamfunction for all densities


100%|██████████| 100/100 [02:45<00:00,  1.66s/it]


saving to so/sf_59_density.pickle
calculating streamfunction for all densities


100%|██████████| 100/100 [02:49<00:00,  1.69s/it]
